### CSV and JSON Creator

This Python script will:

- Run a sql query against Big Query
- Save the results as a dataframe
- Exported the dataframe to a CSV or JSON to your local computer




### Creating Export Functions

The following  creates the functions required to export to csv or to json. 

The minimal parameters you need to pass through the function is the name of a dataframe/sql code 


In [1]:
import pandas as pd
import json
from datetime import datetime as dt

##########################
### Creating functions ###
##########################

print("--- Creating functions ---")

# Export a dataframe to csv
# Pass the sql_code variable through first then optional filname and export path
def export_to_csv(sql_variable, filename = 'python csv export', path = 'C:\\Python CSV Output\\'):
    print('-----------------------')
    print('Start csv export')
    sql_variable = sql_variable
    print('- Creating dataframe from sql code')
    df = pd.read_gbq(sql_variable,
                     project_id = 'data-science-retail',
                     dialect    = 'standard')  
    filename = filename + '-' + str(dt.now().strftime("%Y%m%d-%H%M%S"))
    print('- Filename: '+ filename)
    path = path+filename+'.csv'
    print('- Save location: ' + path)
    df.to_csv(path, index = False, encoding="utf8")
    print('- Successfully exported file - '+filename)
    print('-----------------------')

print("--- Created function 'export_to_csv' ---")   


def export_to_json(dataframe, filename = 'python json export', path = 'C:\\Python CSV Output\\', orient="records", indent=2):
    print('-----------------------')
    print('Start csv export')
    filename = filename + '-' + str(dt.now().strftime("%Y%m%d-%H%M%S"))
    print('- Filename: '+ filename)
    path = path+filename+'.json'
    print('- Save location: ' + path)
    orient = orient
    print('- json orientation = '+orient)
    indent = indent
    dataframe.to_json(path, orient=orient, date_format='iso', indent=indent)
    print('- Successfully exported json file - '+filename)
    print('-----------------------')

print("--- Created function 'export_to_json' ---")   

--- Creating functions ---
--- Created function 'export_to_csv' ---
--- Created function 'export_to_json' ---


In [2]:
###################################
### Creating SQL Code Variables ###
###################################

print('--- Creating SQL variables for each output ---')

# Main SQL code block 
# Create a CTE for each different output file 

main_code = """
WITH
Master_Table_With_True_Comms_Type AS
      (
      SELECT 
            *
            ,'POST' AS Comms_Type_1
            ,CASE
              WHEN Has_PSR_Audio_Comms = 1                 THEN 'PSR Audio Comms'
              WHEN Has_PSR_Braille_Comms = 1               THEN 'PSR Braille Comms'
              WHEN Has_PSR_Largeprint_Blackwhite_Comms = 1 THEN 'PSR Large BW Comms'
              WHEN Has_PSR_Print_Blackwhite_Comms = 1      THEN 'PSR BW Comms'
              WHEN Has_PSR_Largeprint_Comms = 1            THEN 'PSR Largeprint Comms'
              WHEN Has_PSR_Print_Comms = 1                 THEN 'PSR Regular Print Comms' 
              ELSE 'Regular Comms'
             END Comms_Type_2  

      FROM
            `data-science-retail.switching_compensation.phase_3_full_data_snapshot_20201110`

      WHERE
            Account_No IN (SELECT * FROM `data-science-retail.switching_compensation.phase_3_email_bounces`)
      )      

,Staging AS
(
SELECT
      Account_No
     ,Account_Status
     ,Account_Supply_End_Date
     ,Retailer
     ,Payment_Method_Category
     ,Current_Billing_System
     ,Was_CoS_Switching_Overcharge
     ,Was_Renewals_Switching_Overcharge
     ,Total_Overcharge
     ,Total_Goodwill_Inc_PSR_Bonus
     ,Original_Phase1_Goodwill
     ,New_Goodwill_Minus_Original
     ,Total_Refund_Due
     ,Title
     ,CASE WHEN First_Name like '__/__/____' THEN NULL ELSE First_Name END AS First_Name
     ,Last_Name
     ,Has_Tracked_and_Traced_Address
     -- Case statements to use track and trace addresses if we have them, otherwise use the Apollo address data
     ,CASE WHEN Has_Tracked_and_Traced_Address = 1 THEN UPPER(Tracked_Address_Line_1) ELSE UPPER(REPLACE(Address_Line_1, '?','')) END AS Address_Line_1	-- Removing odd question marks from address data
     ,CASE WHEN Has_Tracked_and_Traced_Address = 1 THEN UPPER(Tracked_Address_Line_2) ELSE UPPER(Address_Line_2) END AS Address_Line_2	
     ,CASE WHEN Has_Tracked_and_Traced_Address = 1 THEN UPPER(Tracked_Address_Line_3) ELSE NULL                  END AS Address_Line_3
     ,CASE WHEN Has_Tracked_and_Traced_Address = 1 THEN UPPER(Tracked_Address_Line_4) ELSE UPPER(Town)           END AS Town	
     ,CASE WHEN Has_Tracked_and_Traced_Address = 1 THEN UPPER(Tracked_postcode)       ELSE UPPER(Postcode)       END AS Postcode
     ,Is_PSR
     ,Has_PSR_Comms_Pref
     ,Has_PSR_Audio_Comms
     ,Has_PSR_Braille_Comms
     ,Has_PSR_Print_Comms
     ,Has_PSR_Print_Blackwhite_Comms
     ,Has_PSR_Largeprint_Comms
     ,Has_PSR_Largeprint_Blackwhite_Comms
     -- Comms Type 1 = EMAIL or POST
     ,Comms_Type_1
     -- Comms Type 2 = PSR variant
     ,Comms_Type_2
     ,Comms_Scenario

FROM 
      Master_Table_With_True_Comms_Type     
)
"""

## Combining the main code with an individual SELECT * FROM xxxxx for each CTE you want to export, this will be

sc1_paym = main_code+"""
SELECT * FROM Staging WHERE  Comms_Scenario = 1 AND Payment_Method_Category = 'PAYM'
"""

sc2_paym = main_code+"""
SELECT * FROM Staging WHERE  Comms_Scenario = 2 AND Payment_Method_Category = 'PAYM'
"""

sc3_payg = main_code+"""
SELECT * FROM Staging WHERE  Comms_Scenario = 3 AND Payment_Method_Category = 'PAYG'
"""

sc3_paym = main_code+"""
SELECT * FROM Staging WHERE  Comms_Scenario = 3 AND Payment_Method_Category = 'PAYM'
"""

sc4_payg = main_code+"""
SELECT * FROM Staging WHERE  Comms_Scenario = 4 AND Payment_Method_Category = 'PAYG'
"""

sc4_paym = main_code+"""
SELECT * FROM Staging WHERE  Comms_Scenario = 4 AND Payment_Method_Category = 'PAYM'
"""

## If you get an error here try printing out the combined script (in a different cell) and troubleshoot in Big Query
## e.g:
## print(select_from_cte_1_code)


print("--- Created SQL variables ---")

--- Creating SQL variables for each output ---
--- Created SQL variables ---


In [4]:

###########################
### creating dataframes ###
###########################

print('--- Creating dataframes for each output ---')

## Saving the results of SQL code as a dataframe, 
## Copy and paste this and just change the dataframe and SQL code variable each time

sc1_paym_df = pd.read_gbq(sc1_paym,
                                   project_id = 'data-science-retail',
                                   dialect    = 'standard')

sc2_paym_df = pd.read_gbq(sc2_paym,
                                   project_id = 'data-science-retail',
                                   dialect    = 'standard')

sc3_payg_df = pd.read_gbq(sc3_payg,
                                   project_id = 'data-science-retail',
                                   dialect    = 'standard')

sc3_paym_df = pd.read_gbq(sc3_paym,
                                   project_id = 'data-science-retail',
                                   dialect    = 'standard')

sc4_payg_df = pd.read_gbq(sc4_payg,
                                   project_id = 'data-science-retail',
                                   dialect    = 'standard')

sc4_paym_df = pd.read_gbq(sc4_paym,
                                   project_id = 'data-science-retail',
                                   dialect    = 'standard')

print("--- Created Dataframes ---")

################################################
### Saving Each Dataframes To A Separate CSV ###
################################################

print('--- Starting CSV / JSON export ---')

## This saves the results of the code to the specified path
## You can change the filename by replacing the 'Sample_Filename_1'

export_to_csv(sql_variable = sc1_paym
              ,filename = 'sc1_paym')

export_to_csv(sql_variable = sc2_paym
              ,filename = 'sc2_paym')

export_to_csv(sql_variable = sc3_payg
              ,filename = 'sc3_payg')

export_to_csv(sql_variable = sc3_paym
              ,filename = 'sc3_paym')

export_to_csv(sql_variable = sc4_payg
              ,filename = 'sc4_payg')

export_to_csv(sql_variable = sc4_paym
              ,filename = 'sc4_paym')

print('--- Exports Completed ---')


--- Creating dataframes for each output ---


Downloading: 100%|██████████| 25/25 [00:00<00:00, 101.62rows/s]


--- Created Dataframes ---
--- Starting CSV / JSON export ---
-----------------------
Start csv export
- Creating dataframe from sql code


Downloading: 100%|██████████| 254/254 [00:00<00:00, 762.72rows/s]


- Filename: sc1_paym-20201201-103146
- Save location: C:\Python CSV Output\sc1_paym-20201201-103146.csv
- Successfully exported file - sc1_paym-20201201-103146
-----------------------
-----------------------
Start csv export
- Creating dataframe from sql code


...

- Filename: sc3_paym-20201201-103151
- Save location: C:\Python CSV Output\sc3_paym-20201201-103151.csv
- Successfully exported file - sc3_paym-20201201-103151
-----------------------
-----------------------
Start csv export
- Creating dataframe from sql code


Downloading: 100%|██████████| 85/85 [00:00<00:00, 333.31rows/s]


- Filename: sc4_payg-20201201-103152
- Save location: C:\Python CSV Output\sc4_payg-20201201-103152.csv
- Successfully exported file - sc4_payg-20201201-103152
-----------------------
-----------------------
Start csv export
- Creating dataframe from sql code


Downloading: 100%|██████████| 25/25 [00:00<00:00, 124.37rows/s]

- Filename: sc4_paym-20201201-103154
- Save location: C:\Python CSV Output\sc4_paym-20201201-103154.csv
- Successfully exported file - sc4_paym-20201201-103154
-----------------------
--- Exports Completed ---
